# QoL Index Dashboard

`Nika Karimi Jashni, Tie Ma, Yunhan Liu` <br>

Last Updated: June 6, 2024

In [2]:
import dash
import dash_core_components as dcc
import dash_html_components as html
import pandas as pd
import plotly.express as px
from dash.dependencies import Input, Output

# Sample data: Replace this with your actual data
data = {
    'City': ['Toronto', 'Vancouver', 'Montreal', 'Calgary', 'Ottawa'],
    'Latitude': [43.7, 49.2, 45.5, 51.0, 45.4],
    'Longitude': [-79.4, -123.1, -73.6, -114.1, -75.7],
    'Value1': [1, 2, 3, 4, 5],
    'Value2': [5, 4, 3, 2, 1],
    'Value3': [2, 3, 4, 1, 5],
    'Value4': [3, 1, 2, 5, 4],
    'Value5': [4, 5, 1, 3, 2]
}
df = pd.DataFrame(data)

# Initialize the Dash app
app = dash.Dash(__name__)

# Layout of the dashboard
app.layout = html.Div([
    html.H1("Canada City Data Points"),
    html.Div([
        dcc.Slider(
            id='slider-1',
            min=0,
            max=1,
            step=0.1,
            value=0.2,
            marks={i/10: str(i/10) for i in range(0, 11)}
        ),
        dcc.Slider(
            id='slider-2',
            min=0,
            max=1,
            step=0.1,
            value=0.2,
            marks={i/10: str(i/10) for i in range(0, 11)}
        ),
        dcc.Slider(
            id='slider-3',
            min=0,
            max=1,
            step=0.1,
            value=0.2,
            marks={i/10: str(i/10) for i in range(0, 11)}
        ),
        dcc.Slider(
            id='slider-4',
            min=0,
            max=1,
            step=0.1,
            value=0.2,
            marks={i/10: str(i/10) for i in range(0, 11)}
        ),
        dcc.Slider(
            id='slider-5',
            min=0,
            max=1,
            step=0.1,
            value=0.2,
            marks={i/10: str(i/10) for i in range(0, 11)}
        ),
    ], style={'display': 'grid', 'grid-template-columns': 'repeat(5, 1fr)', 'gap': '10px'}),
    dcc.Graph(id='map')
])

@app.callback(
    Output('map', 'figure'),
    [Input('slider-1', 'value'),
     Input('slider-2', 'value'),
     Input('slider-3', 'value'),
     Input('slider-4', 'value'),
     Input('slider-5', 'value')]
)
def update_map(weight1, weight2, weight3, weight4, weight5):
    # Calculate weighted average
    df['WeightedAvg'] = (weight1 * df['Value1'] + 
                         weight2 * df['Value2'] + 
                         weight3 * df['Value3'] + 
                         weight4 * df['Value4'] + 
                         weight5 * df['Value5'])

    # Create the map
    fig = px.scatter_mapbox(df, lat="Latitude", lon="Longitude", size="WeightedAvg", 
                            hover_name="City", 
                            mapbox_style="carto-positron", 
                            size_max=15, zoom=3)
    
    return fig

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)
